In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import os
import math
import time

#pytorch utility imports
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, TensorDataset
from torchvision.utils import make_grid

#neural net imports
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

In [ ]:
test_df = pd.read_csv('../input/mnist-in-csv/mnist_test.csv')
train_df = pd.read_csv('../input/mnist-in-csv/mnist_train.csv')
train_labels = train_df['label'].values # converting to numpy also
test_labels=test_df['label'].values
train_images = (train_df.iloc[:,1:].values).astype('float32')
test_images = (test_df.iloc[:,1:].values).astype('float32')
train_images = train_images[0:20000]
train_labels = train_labels[0:20000]
print("train images shape",train_images.shape)
print("train labels shape",train_labels.shape)
print("test images shape",test_images.shape)
print("test labels shape",test_labels.shape)

In [ ]:
train_images = train_images.reshape(train_images.shape[0], 28, 28)
test_images = test_images.reshape(test_images.shape[0], 28, 28)
print(train_images.shape)
print(test_images.shape)

#train samples
for i in range(6, 9):
    plt.subplot(330 + (i+1))
    plt.imshow(train_images[i].squeeze(), cmap=plt.get_cmap('gray'))
    plt.title(train_labels[i])

In [ ]:
#test samples
for j in range(3, 9):
    plt.subplot(330 + (j+1))
    plt.imshow(test_images[j].squeeze(), cmap=plt.get_cmap('gray'))

In [ ]:
train_images_tensor = torch.tensor(train_images)/255.0 #default torch.FloatTensor
train_labels_tensor = torch.tensor(train_labels)
train_tensor = TensorDataset(train_images_tensor, train_labels_tensor)

test_images_tensor = torch.tensor(test_images)/255.0
test_labels_tensor = torch.tensor(test_labels)
test_tensor = TensorDataset(test_images_tensor, test_labels_tensor)
train_loader = DataLoader(train_tensor, batch_size=16, num_workers=2, shuffle=True)
test_loader = DataLoader(test_images_tensor, batch_size=16, num_workers=2, shuffle=False)

In [ ]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        
        self.fc1 = nn.Linear(784, 548)
        self.bn1 = nn.BatchNorm1d(548)
        
        self.fc2 = nn.Linear(548, 252)
        self.bn2 = nn.BatchNorm1d(252)
        
        self.fc3 = nn.Linear(252, 10)
        
        
    def forward(self, x):
        x = x.view((-1, 784))
        h = self.fc1(x)
        h = self.bn1(h)
        h = F.relu(h)
        h = F.dropout(h, p=0.5, training=self.training)
        
        h = self.fc2(h)
        h = self.bn2(h)
        h = F.relu(h)
        h = F.dropout(h, p=0.2, training=self.training)
        
        h = self.fc3(h)
        out = F.log_softmax(h,dim=1)
        return out

model = Model()
print(model)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

if (device.type=='cuda'):
    model.cuda() # convert model to cuda model

    
optimizer = optim.Adam(model.parameters(), lr=0.001) #adam optimizer from optim module

In [ ]:
model.train()


losses = []
for epoch in range(5):
    for batch_idx, (data, target) in enumerate(train_loader):
        # Get Samples
        if (device.type=='cuda'):
            data, target = Variable(data.cuda()), Variable(target.cuda())
        else:
            data, target = Variable(data), Variable(target) # making group of 16
            
        
        # Init
        optimizer.zero_grad() #making gradient zero for new mini-batch. 

        # Predict
        y_pred = model(data) 
         
        
        # Calculate loss
        loss = F.cross_entropy(y_pred, target)
        losses.append(loss.data)
        
        # Backpropagation
        loss.backward()  #It computes gradient of loss w.r.t all the parameters and store them in (parameter.grad) attribute.
        optimizer.step() #optimizer.step() updates all the parameters based on (parameter.grad)
        
        
        # Display
        #if batch_idx % 100 == 1:
        print('\r Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format( epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data), end='')
            
    print()

In [ ]:
if (device.type=='cuda'):
    evaluate_x=test_images_tensor.cuda()
    evaluate_y=test_labels_tensor.cuda()
else:
    evaluate_x=test_images_tensor
    evaluate_y=test_labels_tensor
    

output = model(evaluate_x)

pred = output.data.max(1)[1]
d = pred.eq(evaluate_y.data).cpu()
a=(d.sum().data.cpu().numpy())
b=d.size()
b=torch.tensor(b)
b=(b.sum().data.cpu().numpy())
accuracy = a/b

print('Accuracy:', accuracy)

In [ ]:
print(evaluate_y[3: 9])
print(pred[3: 9])